In [7]:
import numpy as np
import random
from structures import *
import matplotlib.pyplot as plt

In [13]:
TIMESTEPS = 60    # 1 timestep = 5 min
N_STATIONS = 20
DOCKS_PER_STATION = 10
INIT_BIKES_PER_STATION = 5
RESERVATION_LIMIT = 5

RIDE_DENSITY_AVG = 10
RIDE_DENSITY_STD = 2
RIDE_LEN_AVG = 4
RIDE_LEN_STD = 3


def run_simulation():
    rides_log = []
    failed_checkout_log = []
    failed_dock_log = []
    
    
    stations = []
    current_rides = []
    
    # create stations
    for station in range(N_STATIONS):
        # currently no EBIKES
        stations.append(Station(DOCKS_PER_STATION, INIT_BIKES_PER_STATION, 0, RESERVATION_LIMIT))
    
    # run simulation!
    for time in range(TIMESTEPS):
        n_checkout_fails = 0
        n_dock_fails = 0
        
        # start new rides
        n_rides_started = sample_norm_int(RIDE_DENSITY_AVG, RIDE_DENSITY_STD, 0)
        for _ in range(n_rides_started):
            start = random.choice(stations)
            destination = random.choice(stations)
            length = sample_norm_int(RIDE_LEN_AVG, RIDE_LEN_STD, 1)
    
            # try to check out bike (currently only SBIKEs)
            if start.check_out("SBIKE"):
                # checkout successful
                new_ride = Ride(start, destination, time, length)
                current_rides.append(new_ride)
            else:
                # no bikes available
                n_checkout_fails += 1
        
        # check for rides finishing
        updated_current_rides = []
        for ride in current_rides:
            if ride.start_time + ride.length >= time:   # check if they're done riding
                if ride.destination.return_bike("SBIKE"):  # try to return bike
                    # return successful
                    pass
                else:
                    # return failed
                    updated_current_rides.append(ride)
                    n_dock_fails += 1
            else:
                updated_current_rides.append(ride)
    
        current_rides = updated_current_rides
    
        rides_log.append(len(current_rides))
        failed_checkout_log.append(n_checkout_fails)
        failed_dock_log.append(n_dock_fails)

    return rides_log, failed_checkout_log, failed_dock_log


rides, checkout_fails, dock_fails = run_simulation()

In [ ]:
plt.plot(rides, label="Rides")
plt.plot(checkout_fails, label="Failed checkouts")
plt.plot(dock_fails, label="Failed docks")
plt.legend()